In [ ]:
# 目标：构建具有“记忆”的聊天机器人
# 对话中模型具有上下文记忆，多用户对话
# 对话历史记录管理（消息过长裁剪、保留system消息）
# 对话信息持久化

In [ ]:

# 相关环境变量设置
import config_loader

config_loader.load_env()

In [ ]:
# 我们单次调用模型聊天，它是不会记住上次聊了什么的
# 我们想让他知道之前的对话记录，就需要把之前的对话记录发给他，然后附上最新的问题
# 这样就让模型有了“记忆”的效果
# 参考如下示例
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash")
ai_msg = llm.invoke("我的名字是张三")
print(ai_msg.content)

print("="*100)
ai_msg = llm.invoke("我是谁？")
# 并不会记住我的名字
print(ai_msg.content)

In [ ]:
# 我们需要把之前的对话记录全部都发送给LLM 就会有记忆效果
from langchain_core.messages.human import HumanMessage
from langchain_core.messages.ai import AIMessage


msg_list = [
    HumanMessage(content="我的名字是张三"),
    AIMessage(content="你好，张三！很高兴认识你。有什么我可以帮你的吗？"),
    HumanMessage(content="我是谁？")
]
llm.invoke(msg_list).content

In [ ]:
# 上面是模型记忆的原理，我们需要持久化对话记录，自动封装之前的上下文
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

# 定义一个 graph
workflow = StateGraph(state_schema=MessagesState)

# 定义一个调用模型的函数
def call_model(state: MessagesState):
    # 可以观察这行日志，lang graph 会自动拼接之前的对话记录封装到 MessageState 对象中
    print("当前 MessageState: ", state)
    print("当前 messages: ", state["messages"])
    response = llm.invoke(state["messages"])
    return {"messages": response}

# 定义 graph 中的（单个）节点
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# 添加记忆存储 这个支持内存、sqlite、postgres，参考：https://langchain-ai.github.io/langgraph/concepts/persistence/#checkpointer-libraries
# 默认是内存存储数据，生产环境推荐 postgres
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

# 配置当前用户线程ID 这样单个app 就能支持多个对话线程
config = {"configurable": {"thread_id": "abc123"}}

query = "你好，我是张三"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()


In [ ]:
query = "我叫什么名字？"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

In [ ]:
# 换个对话线程id 就不会把其他对话线程id给带出来
config = {"configurable": {"thread_id": "aaa"}}

query = "我叫什么名字？"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

In [ ]:
# 但是还是用原来的线程id 就还保留着之前的记忆
config = {"configurable": {"thread_id": "abc123"}}

query = "我叫什么名字？"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

In [ ]:
# 添加提示词模板
# 上面介绍了对话如何带有上下文记忆，现在介绍对话的时候如何带有提示词模板
# 比如说现在需要预输入一个系统提示词，然后并带有“语言”参数
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages.system import SystemMessage

# MessagesPlaceholder 传递所有消息
# 预输入一个系统提示词
prompt_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content="你是一名老中医，尽你最大的能力用 {language} 去回答所有问题"),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [ ]:
from typing import Sequence
from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from typing_extensions import Annotated, TypedDict

# 因为我们还有个 language 参数，所以我们需要自定义state约束
class State(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    # 定义 language 属性和属性类型
    language: str

workflow = StateGraph(state_schema=State)

def call_model(state: State):
    # 这里对 state 进行模板转换
    prompt = prompt_template.invoke(state)
    print("当前 prompt: ", prompt)
    response = llm.invoke(prompt)
    return {"messages": response}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [ ]:
config = {"configurable": {"thread_id": "test-with-prompt"}}
query = "Hi! I'm zhangsan"
language = "中文"

input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

In [ ]:
# 因为整个状态都是持久的，所以language参数没有变动的时候，可以不传参
query = "我是谁？"
input_messages = [HumanMessage(query)]

output = app.invoke(
    {"messages": input_messages},
    config,
)
output["messages"][-1].pretty_print()

In [ ]:
# 新问题：大模型可接受的上下文长度终归是有限的，总不能把所有的对话历史记录都传递给它吧
# 所以就有了限制传入消息的大小操作 这里用 trim_messages 来减少发送给模型的消息数量
# 它允许我们指定要保留多少个标记，例如始终保留system消息和允许部分human消息
from langchain_core.messages import SystemMessage, trim_messages

trimmer = trim_messages(
    # 限制消息的最大token长度
    max_tokens=65,
    strategy="last",
    token_counter=llm,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

messages = [
    SystemMessage(content="你是一名老中医，专治吹牛*"),
    HumanMessage(content="你好，我是张三，以前和秦始皇掰过手腕"),
    AIMessage(content="我信你"),
    HumanMessage(content="我上周去火星转了一圈，发现没有西兰花，所以我回来了"),
    AIMessage(content="厉害"),
    HumanMessage(content="你说 1 + 1 等于几？"),
    AIMessage(content="老夫认为，如果是合作共赢的话，两个人合作会出现1+1>2的效果！"),
    HumanMessage(content="我谢谢你"),
    AIMessage(content="包的"),
    HumanMessage(content="你快乐吗？"),
    AIMessage(content="快乐"),
]

# 运行结果可以看到，根据配置，保留了system消息，和在最大token的限制下，移除了早期的聊天记录
trimmer.invoke(messages)

In [ ]:
# 实际使用这个，其实只需要在调用模型之前，invoke 一下，把messages裁剪一下就行
workflow = StateGraph(state_schema=State)


def call_model(state: State):
    trimmed_messages = trimmer.invoke(state["messages"])
    print("裁剪后的messages: ", trimmed_messages)
    prompt = prompt_template.invoke(
        {"messages": trimmed_messages, "language": state["language"]}
    )
    response = llm.invoke(prompt)
    return {"messages": [response]}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [ ]:
config = {"configurable": {"thread_id": "test-with-prompt"}}
query = "我是谁？"
language = "中文"

# 把之前超过token的messages测试数据也带上
input_messages = messages + [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
# 根据输出信息，可以发现裁剪有效！
output["messages"][-1].pretty_print()

In [ ]:
# 如何流式输出？
# 直接调用 stream 方法即可 参数与 invoke 类似
config = {"configurable": {"thread_id": "test-with-prompt"}}
query = "我是谁？"
language = "中文"

# 把之前超过token的messages测试数据也带上
input_messages = messages + [HumanMessage(query)]

for chunk, metadata in app.stream(
    {"messages": input_messages, "language": language},
    config,
    # 加上流式输出模式
    stream_mode="messages",
):
    if isinstance(chunk, AIMessage):
        print(chunk.content, end="|")